In [8]:
from ultralytics import YOLO
import os

model_path = 'yolo/bee_cam/training_run_2025-06-02_09-17-572/weights/best.pt'
# model_path = '../bee_cam_2024/beecam_2024_final.pt'
model = YOLO(model_path)



In [2]:
# RUN ON ALL IMGS

results = model.predict(
    source='yolo/datasets/val_tps',
    save=True,              
    save_txt=False,
    visualize=False,         
    conf=0.25,             
    imgsz=640,              
    device=0                
)



image 1/712 /home/mpgetz/repos/bee_cam_2025/yolo/datasets/val_tps/6502_t4_hj_20230719_3.jpg: 640x640 1 insect, 3.4ms
image 2/712 /home/mpgetz/repos/bee_cam_2025/yolo/datasets/val_tps/6504_t4_hj_20230719_4.jpg: 640x640 1 insect, 3.5ms
image 3/712 /home/mpgetz/repos/bee_cam_2025/yolo/datasets/val_tps/6508_t4_hj_20230719_4.jpg: 640x640 1 insect, 3.4ms
image 4/712 /home/mpgetz/repos/bee_cam_2025/yolo/datasets/val_tps/6512_t4_hj_20230719_4.jpg: 640x640 1 insect, 3.4ms
image 5/712 /home/mpgetz/repos/bee_cam_2025/yolo/datasets/val_tps/6516_t4_hj_20230719_4.jpg: 640x640 1 insect, 3.4ms
image 6/712 /home/mpgetz/repos/bee_cam_2025/yolo/datasets/val_tps/6520_t4_hj_20230719_9.jpg: 640x640 1 insect, 3.4ms
image 7/712 /home/mpgetz/repos/bee_cam_2025/yolo/datasets/val_tps/6524_t4_hj_20230719_4.jpg: 640x640 1 insect, 3.4ms
image 8/712 /home/mpgetz/repos/bee_cam_2025/yolo/datasets/val_tps/6528_t4_hj_20230719_7.jpg: 640x640 1 insect, 3.4ms
image 9/712 /home/mpgetz/repos/bee_cam_2025/yolo/datasets/val_t

In [7]:
# ONLY SAVE FNs

from pathlib import Path
from PIL import Image
import shutil

input_dir = Path('/home/mpgetz/nas/bee_cam/2024_img_processing/bombus_apis_clean/bombus_imgs_clean')
# input_dir = Path('yolo/datasets/val_tps')

output_dir = Path('runs/val_tps_2024_model')
output_dir.mkdir(parents=True, exist_ok=True)

for img_path in input_dir.glob('*.[jp][pn]g'):
    results = model(img_path, conf=0.25, imgsz=640, device=0, verbose=False)
    has_detections = any(len(r.boxes) > 0 for r in results)

    if not has_detections:
        shutil.copy2(img_path, output_dir / img_path.name)

In [4]:
from sahi import AutoDetectionModel
from sahi.predict import get_prediction
from sahi.predict import get_sliced_prediction
from sahi.predict import predict

input_dir = Path("/home/mpgetz/nas/bee_cam/2024_img_processing/bombus_apis_clean/bombus_imgs_clean")
output_dir = Path("runs/bombus_no_detections_1")
output_dir.mkdir(exist_ok=True, parents=True)

detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path=model_path,
    confidence_threshold=0.3,
    device="cuda:0"
)

for img_path in input_dir.glob("*.[jp][pn]g"):
    result = get_sliced_prediction(
        image=str(img_path),
        detection_model=detection_model,
        slice_height=640,
        slice_width=640,
        overlap_height_ratio=0.25,
        overlap_width_ratio=0.25,
    )

    if len(result.object_prediction_list) == 0:
        shutil.copy2(img_path, output_dir / img_path.name)

# image_path = "jpgs_to_annot/pi1_20240718_130459.jpg"
# result = get_sliced_prediction(
#     image_path,
#     detection_model,
#     slice_height=640,
#     slice_width=640,
#     overlap_height_ratio=0.25,
#     overlap_width_ratio=0.25
# )

# result.export_visuals(export_dir="output_sahi")

# predict(
#     model_type="ultralytics",
#     model_path=model_path,
#     model_device="cuda", 
#     model_confidence_threshold=0.3,
#     source="jpgs_to_annot",
#     slice_height=640,
#     slice_width=640,
#     overlap_height_ratio=0.25,
#     overlap_width_ratio=0.25,
# )

Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 

In [ ]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from pathlib import Path
from PIL import Image

# Setup
input_dir = Path("/home/mpgetz/nas/bee_cam/2024_img_processing/bombus_apis_clean/bombus_imgs_clean")
output_txt = Path("all_bombus_predictions.txt")

# Load model
detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path=model_path,  # define this before running
    confidence_threshold=0.4,
    device="cuda:0"
)

# Open output file for writing
with open(output_txt, "w") as out_file:
    for img_path in input_dir.glob("*.[jp][pn]g"):
        # Get image dimensions for normalization
        with Image.open(img_path) as img:
            img_width, img_height = img.size

        result = get_sliced_prediction(
            image=str(img_path),
            detection_model=detection_model,
            slice_height=640,
            slice_width=640,
            overlap_height_ratio=0.25,
            overlap_width_ratio=0.25,
        )

        for pred in result.object_prediction_list:
            # Get absolute bbox: [x1, y1, x2, y2]
            x1, y1, x2, y2 = pred.bbox.to_xyxy()

            # Convert to YOLO format: [x_center, y_center, width, height], all normalized
            x_center = ((x1 + x2) / 2) / img_width
            y_center = ((y1 + y2) / 2) / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height

            class_id = 0

            out_file.write(f"{img_path.name} {class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 slices.
Performing prediction on 15 